<a href="https://colab.research.google.com/github/manuelboi/MLsec_project/blob/main/MLsec_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Models:
1. Fixing Data Augmentation to Improve Adversarial Robustness
2. Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?
3. MMA Training: Direct Input Space Margin Maximization through Adversarial Training

# General imports

In [6]:
%%capture

# Various
import matplotlib.pyplot as plt
import time
import importlib.util
import re

# Pytorch
import torch
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import StepLR

# RobustBench
!pip install git+https://github.com/RobustBench/robustbench
from robustbench.utils import load_model
from robustbench.eval import benchmark
from robustbench.data import load_cifar10
from robustbench.model_zoo.enums import ThreatModel

# Smoothing
!git clone https://github.com/matteoturnu/smoothing.git
# smoothing repository dependencies
!conda create -n smoothing
!conda activate smoothing
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
!conda install scipy pandas statsmodels matplotlib seaborn
!pip install setGPU
from smoothing.code.core import Smooth
from smoothing.code.train import train, test
from smoothing.code.train_utils import AverageMeter, accuracy

IndentationError: expected an indented block after function definition on line 102 (core.py, line 103)

# Setup

In [2]:
labels_dct = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}
# use GPU if available, otherwise use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Computing device used: ", device)

# Preparing trainset and testset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define the number of samples you want to take
num_samples = 100
# Create a subset of the dataset
indices = list(range(num_samples))
train_subset = Subset(trainset, indices)
test_subset = Subset(testset, indices)

# Preparing trainloader and testloader
trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=2)

# Load test samples for predict and certify
x_test, y_test = load_cifar10(10)
x_test, y_test = x_test.to(device), y_test.to(device)

# Setting various parameters
epochs = 1 # used for training
sigma = 0.5 # gaussian noise standard deviation
n_examples = 5 # to be perturbed by AutoAttack
eps_L2 = 0.5 # epsilon of the perturbancy for L2 norm
eps_Linf = 8/255 # epsilon of the perturbancy with Linf norm

Computing device used:  cpu


100%|██████████| 170498071/170498071 [00:02<00:00, 59107897.61it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


# Functions

Model training

In [3]:
def train_model(trainloader, testloader, model, epochs, sigma, device):
  criterion = CrossEntropyLoss().to(device)
  optimizer = SGD(model.parameters())
  scheduler = StepLR(optimizer, step_size=30)

  for epoch in range(epochs):
    scheduler.step(epoch)
    before = time.time()
    train_loss, train_acc = train(trainloader, model, criterion, optimizer, epoch, sigma, device)
    test_loss, test_acc = test(testloader, model, criterion, epoch, sigma, device)
    after = time.time()

  return model

Smooth model performances on perturbed images

In [4]:
def certify(model, sigma, x_text, y_test):
  n_classes = 10
  alpha = 0.05 # (1 - alpha) is the confidence level (in this case is 0.95)
  n0 = 10 # number of samples for selection
  n = 100 # number of samples for estimation (certify) (too few samples but computation time is strongly affected with more)

  smooth_model = Smooth(model, n_classes, sigma)

  top_classes = list()
  radiuses = list()
  for x, y in zip(x_test, y_test):
    top_class = smooth_model.predict(x, n0, alpha, batch_size=n0, device=device)
    top_class, radius = smooth_model.certify(x, n0, n, alpha, batch_size=n0, device=device)
    top_classes.append(top_class)
    radiuses.append(radius)

  top_classes = torch.tensor(top_classes, dtype=torch.float64)
  accuracy = torch.mean(top_classes == y_test, dtype=torch.float64)

  print("Top classes: ", top_classes)
  print("Y classes: ", y_test)
  print("Accuracy on smooth model: ", accuracy)

# Fixing Data Augmentation to Improve Adversarial Robustness

L2

In [ ]:
# Model loading
model_1_L2 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='L2')
model_1_L2.to(device)

# Model training
model_1_L2 = train_model(trainloader, testloader, model_1_L2, epochs, sigma, device, 'L2')

# Model prediction and certification
certify(model_1_L2, sigma, x_test, y_test)

# AutoAttack on model 1 with L2
benchmark(model_1_L2, threat_model=ThreatModel.L2, n_examples=n_examples, eps=eps_L2)

Linf

In [ ]:
# Model loading
model_1_Linf = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf')
model_1_Linf.to(device)

# Model training
model_1_Linf = train_model(trainloader, testloader, model_1_Linf, epochs, sigma, device, 'Linf')

# Model prediction and certification
certify(model_1_Linf, sigma, x_test, y_test)

# AutoAttack on model 1 with Linf
benchmark(model_1_Linf, threat_model=ThreatModel.Linf, n_examples=n_examples, eps=eps_Linf)

# Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?

L2

In [ ]:
# Model loading
model_2_L2 = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='L2')
model_2_L2.to(device)

# Model training
model_2_L2 = train_model(trainloader, testloader, model_2_L2, epochs, sigma, device, 'L2')

# Model prediction and certification
certify(model_2_L2, sigma, x_test, y_test)

# AutoAttack on model 2 with L2
benchmark(model_2_L2, threat_model=ThreatModel.L2, n_examples=n_examples, eps=eps_L2)

Linf

In [ ]:
# Model loading
model_2_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
model_2_Linf.to(device)

# Model training
model_2_Linf = train_model(trainloader, testloader, model_2_Linf, epochs, sigma, device, 'Linf')

# Model prediction and certification
certify(model_2_Linf, sigma, x_test, y_test)

# AutoAttack on model 2 with Linf
benchmark(model_2_Linf, threat_model=ThreatModel.Linf, n_examples=n_examples, eps=eps_Linf)

# MMA Training: Direct Input Space Margin Maximization through Adversarial Training

L2

In [ ]:
# Model loading
model_3_L2 = load_model(model_name='Ding2020MMA', dataset='cifar10', threat_model='L2')
model_3_L2.to(device)

# Model training
model_3_L2 = train_model(trainloader, testloader, model_3_L2, epochs, sigma, device, 'L2')

# Model prediction and certification
certify(model_3_L2, sigma, x_test, y_test)

# AutoAttack on model 3 with L2
benchmark(model_3_L2, threat_model=ThreatModel.L2, n_examples=n_examples, eps=eps_L2)

Linf

In [ ]:
# Model loading
model_3_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
model_3_Linf.to(device)

# Model training
model_3_Linf = train_model(trainloader, testloader, model_3_Linf, epochs, sigma, device, 'Linf')

# Model prediction and certification
certify(model_3_Linf, sigma, x_test, y_test)

# AutoAttack on model 3 with Linf
benchmark(model_3_Linf, threat_model=ThreatModel.Linf, n_examples=n_examples, eps=eps_Linf)